# Merging the data: Investigation of anomalous undervote rate in Lt. Gov contest

## P.B. Stark and Kellie Ottoboni

H0: number of undervotes is uncorrelated with the percent of minority voters

In [1]:
%matplotlib inline
import math
import numpy as np
import scipy as sp
import scipy.optimize
from scipy.stats import hypergeom, binom, norm, chi2
from scipy import special
from cryptorandom.cryptorandom import SHA256
from cryptorandom import sample
from permute.core import corr
from permute.npc import fisher
import matplotlib.pyplot as plt
import pandas as pd
import csv

reps = 100

In [2]:
statewide_contests = np.array(["Governor", "Lieutenant Governor", "Secretary Of State", \
                               "Attorney General", "Commissioner Of Agriculture", \
                               "Commissioner Of Insurance", "State School Superintendent", "Commissioner Of Labor"])

votes = pd.read_csv('../../Data/County_votes/undervotes_by_precinct.csv')
votes.head()

,County,Precinct_ID,Vote type,Total ballots,Contest,Ballots cast,Undervotes
0,APPLING,1b,Absentee by Mail,70,Governor,70,0
1,APPLING,1b,Advance in Person,375,Governor,375,0
2,APPLING,1b,Election Day,335,Governor,335,0
3,APPLING,1b,Provisional,2,Governor,2,0
4,APPLING,1c,Absentee by Mail,37,Governor,37,0


In [3]:
votes["Precinct_number"]=votes.Precinct_ID.str.extract(r'([a-z]?\d+[a-z]*[-]*\d*)')

#votes.Precinct_ID.str.extract(r'(\d+)') # create column for precinct id NUMBERS
votes["Precinct_word"]=votes.Precinct_ID.str.extract(r'([a-z]+\s?.*)') # column for word name



In [4]:
turnout = pd.read_csv('../../Data/turnout_by_precinct.csv')
turnout["Precinct_name"] = turnout["Precinct_name"].str.strip()

# Fix Chatham county precincts 407-c and 408-c, which have duplicated IDs
turnout = turnout.replace({"Precinct_ID":{"4-07c":"4-07", "4-08c":"4-08"}})

# Drop unusual empty-ish precincts

turnout = turnout.drop( [
    turnout.loc[(turnout.Precinct_ID == "jb11") & (turnout.County=="CHATHAM")].index[0],
    turnout.loc[(turnout.Precinct_ID == "132") & (turnout.County=="BARROW")].index[0],
    turnout.loc[(turnout.Precinct_ID == "ch03") & (turnout.County=="EFFINGHAM")].index[0],
    turnout.loc[(turnout.Precinct_ID == "111") & (turnout.County=="FORSYTH")].index[0],
    turnout.loc[(turnout.Precinct_ID == "1") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "1000") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "1i") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "4") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "50") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "730") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "md01") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "jb02") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "vg04") & (turnout.County=="FULTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "1714") & (turnout.County=="SPALDING")].index[0],
    turnout.loc[(turnout.Precinct_ID == "31") & (turnout.County=="SPALDING")].index[0],
    turnout.loc[(turnout.Precinct_ID == "57") & (turnout.County=="SPALDING")].index[0],
    turnout.loc[(turnout.Precinct_ID == "3") & (turnout.County=="BROOKS")].index[0],
    turnout.loc[(turnout.Precinct_ID == "421") & (turnout.County=="BRYAN")].index[0],
    turnout.loc[(turnout.Precinct_ID == "13") & (turnout.County=="COBB")].index[0],
    turnout.loc[(turnout.Precinct_ID == "jb13") & (turnout.County=="COBB")].index[0],
    turnout.loc[(turnout.Precinct_ID == "714a1") & (turnout.County=="COWETA")].index[0],
    turnout.loc[(turnout.Precinct_ID == "gf4") & (turnout.County=="DOUGHERTY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "cuba") & (turnout.County=="GRADY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "67") & (turnout.County=="HALL")].index[0],
    turnout.loc[(turnout.Precinct_ID == "154") & (turnout.County=="HALL")].index[0],
    turnout.loc[(turnout.Precinct_ID == "106") & (turnout.County=="HARRIS")].index[0],
    turnout.loc[(turnout.Precinct_ID == "12a") & (turnout.County=="HENRY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "295") & (turnout.County=="HENRY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "gb") & (turnout.County=="HENRY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "le") & (turnout.County=="HENRY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "lj3") & (turnout.County=="HENRY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "ss09b") & (turnout.County=="HENRY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "5") & (turnout.County=="HOUSTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "ni") & (turnout.County=="MURRAY")].index[0],
    turnout.loc[(turnout.Precinct_ID == "418") & (turnout.County=="NEWTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "dg") & (turnout.County=="WALKER")].index[0],
    turnout.loc[(turnout.Precinct_ID == "41") & (turnout.County=="WALTON")].index[0],
    turnout.loc[(turnout.Precinct_ID == "3") & (turnout.County=="WHITE")].index[0]
    #turnout.loc[(turnout.Precinct_ID == "88888")].index,
    #turnout.loc[(turnout.Precinct_ID == "99999")].index
])

# Fix Jefferson county manually
turnout.loc[(turnout.County=="JEFFERSON") & (turnout.Precinct_ID=="1460"), "Precinct_name"] = "stapleton crossroads"

turnout.head()

,County,Precinct_ID,Precinct_name,Black_Male_reg,Black_Male_voted,Black_Female_reg,Black_Female_voted,Black_unknown_reg,Black_unknown_voted,White_Male_reg,...,Other_unknown_reg,Other_unknown_voted,Unknown_Male_reg,Unknown_Male_voted,Unknown_Female_reg,Unknown_Female_voted,Unknown_unknown_reg,Unknown_unknown_voted,Total_reg,Total_voted
0,GLYNN,1713,sterling elementary,206,100,264,165,0,0,1537,...,0,0,218,79,214,80,2,1,4283,2379
1,GLYNN,1923,marshes of glynn,251,134,273,180,1,0,1068,...,0,0,150,57,154,56,1,1,3192,1772
2,GLYNN,1933,satilla marsh school,132,76,198,116,1,0,1003,...,0,0,127,59,142,70,0,0,2724,1574
3,GLYNN,1943,blythe island baptist,52,26,81,46,0,0,971,...,0,0,102,35,102,38,1,1,2423,1450
4,GLYNN,2713,christian renewal,18,8,15,8,0,0,1478,...,0,0,148,76,208,127,5,3,3685,2512


In [5]:
hand_fixed = pd.read_excel('../../Data/kno-rowsneedfixin.xlsx')
hand_fixed.head()


,County,Precinct_ID,Precinct_number,Precinct_word,Precinct_name
0,BAKER,hoggard mill,NaN,hoggard mill,hoggards mill
1,BALDWIN,north mill,NaN,north mill,south milledgeville
2,BALDWIN,south mill,NaN,south mill,north milledgeville
3,BARTOW,woodland high,NaN,woodland high,woodland
4,BLECKLEY,fairground,NaN,fairground,1


In [6]:
votes_fixed = pd.merge(votes, hand_fixed, left_on=["County", "Precinct_ID"], 
                       right_on=["County", "Precinct_ID"], how="left")
votes_fixed = votes_fixed.rename(index=str, columns={"Precinct_ID_x": "Precinct_ID", 
                                       "Precinct_number_x": "Precinct_number",
                                       "Precinct_word_x": "Precinct_word",
                                       "Precinct_name":"Precinct_ID_mod"})

votes_fixed = votes_fixed.replace({"County":
                                   {"Ben_Hill":"BEN HILL",
                                    "Jeff_Davis":"JEFF DAVIS"},
                                   "Precinct_ID_mod":
                                   {1:"1", 100:"100", 
                                    300:"300", 304:"304",
                                    400:"400", 404:"404",
                                    405:"405", 406:"406",
                                    407:"407", 408:"408", 409:"409"}
                                  })
votes_fixed.head()

,County,Precinct_ID,Vote type,Total ballots,Contest,Ballots cast,Undervotes,Precinct_number,Precinct_word,Precinct_number_y,Precinct_word_y,Precinct_ID_mod
0,APPLING,1b,Absentee by Mail,70,Governor,70,0,1b,b,NaN,NaN,NaN
1,APPLING,1b,Advance in Person,375,Governor,375,0,1b,b,NaN,NaN,NaN
2,APPLING,1b,Election Day,335,Governor,335,0,1b,b,NaN,NaN,NaN
3,APPLING,1b,Provisional,2,Governor,2,0,1b,b,NaN,NaN,NaN
4,APPLING,1c,Absentee by Mail,37,Governor,37,0,1c,c,NaN,NaN,NaN


In [19]:
merge1 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_number"], right_on=["County","Precinct_ID"])
merge2 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_word"], right_on=["County", "Precinct_name"])
merge3 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_ID"], right_on=["County","Precinct_ID"])
merge4 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_ID"], right_on=["County","Precinct_name"])
merge5 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_ID_mod"], right_on=["County","Precinct_name"])
merge = pd.concat([merge1, merge2, merge3, merge4, merge5], sort=False).drop_duplicates()
merge.head()

,County,Precinct_ID_x,Vote type,Total ballots,Contest,Ballots cast,Undervotes,Precinct_number,Precinct_word,Precinct_number_y,...,Other_unknown_voted,Unknown_Male_reg,Unknown_Male_voted,Unknown_Female_reg,Unknown_Female_voted,Unknown_unknown_reg,Unknown_unknown_voted,Total_reg,Total_voted,Precinct_ID
0,APPLING,1b,Absentee by Mail,70,Governor,70,0,1b,b,NaN,...,0,48,12,31,5,0,0,1269,782,NaN
1,APPLING,1b,Advance in Person,375,Governor,375,0,1b,b,NaN,...,0,48,12,31,5,0,0,1269,782,NaN
2,APPLING,1b,Election Day,335,Governor,335,0,1b,b,NaN,...,0,48,12,31,5,0,0,1269,782,NaN
3,APPLING,1b,Provisional,2,Governor,2,0,1b,b,NaN,...,0,48,12,31,5,0,0,1269,782,NaN
4,APPLING,1b,Absentee by Mail,70,Lieutenant Governor,68,2,1b,b,NaN,...,0,48,12,31,5,0,0,1269,782,NaN


In [20]:
merge = merge.drop(["Precinct_number", "Precinct_word", "Precinct_number_y", 
                    "Precinct_word_y", "Precinct_ID", "Precinct_ID_y"], axis=1)
merge = merge.drop_duplicates()
merge = merge.rename(index=str, columns={"Precinct_ID_x":"Precinct_ID"})
merge = merge[pd.notnull(merge.Precinct_name)]
merge = merge[pd.notnull(merge.Precinct_ID)]
merge.head()

,County,Precinct_ID,Vote type,Total ballots,Contest,Ballots cast,Undervotes,Precinct_ID_mod,Precinct_name,Black_Male_reg,...,Other_unknown_reg,Other_unknown_voted,Unknown_Male_reg,Unknown_Male_voted,Unknown_Female_reg,Unknown_Female_voted,Unknown_unknown_reg,Unknown_unknown_voted,Total_reg,Total_voted
0,APPLING,1b,Absentee by Mail,70,Governor,70,0,NaN,1b,31,...,0,0,48,12,31,5,0,0,1269,782
1,APPLING,1b,Advance in Person,375,Governor,375,0,NaN,1b,31,...,0,0,48,12,31,5,0,0,1269,782
2,APPLING,1b,Election Day,335,Governor,335,0,NaN,1b,31,...,0,0,48,12,31,5,0,0,1269,782
3,APPLING,1b,Provisional,2,Governor,2,0,NaN,1b,31,...,0,0,48,12,31,5,0,0,1269,782
4,APPLING,1b,Absentee by Mail,70,Lieutenant Governor,68,2,NaN,1b,31,...,0,0,48,12,31,5,0,0,1269,782


In [21]:
merge.loc[(merge.County=="APPLING")&(merge.Contest=="Governor")&(merge["Vote type"]=="Election Day")]

,County,Precinct_ID,Vote type,Total ballots,Contest,Ballots cast,Undervotes,Precinct_ID_mod,Precinct_name,Black_Male_reg,...,Other_unknown_reg,Other_unknown_voted,Unknown_Male_reg,Unknown_Male_voted,Unknown_Female_reg,Unknown_Female_voted,Unknown_unknown_reg,Unknown_unknown_voted,Total_reg,Total_voted
2,APPLING,1b,Election Day,335,Governor,335,0,NaN,1b,31,...,0,0,48,12,31,5,0,0,1269,782
42,APPLING,1c,Election Day,261,Governor,261,0,NaN,1c,9,...,0,0,25,5,31,11,0,0,837,543
82,APPLING,2,Election Day,531,Governor,531,0,NaN,2,576,...,0,0,87,18,65,9,0,0,2279,1163
122,APPLING,3a1,Election Day,292,Governor,292,0,NaN,3a1,6,...,0,0,27,10,20,7,0,0,780,539
162,APPLING,3c,Election Day,320,Governor,320,0,NaN,3c,141,...,0,0,61,10,60,17,0,0,1614,931
202,APPLING,4b,Election Day,314,Governor,314,0,NaN,4b,5,...,0,0,27,7,17,5,0,0,826,588
242,APPLING,4d,Election Day,377,Governor,377,0,NaN,4d,34,...,0,0,49,16,51,15,0,0,1488,975
282,APPLING,5a,Election Day,243,Governor,243,0,NaN,5a,46,...,0,0,51,6,47,12,0,0,1153,637
322,APPLING,5b,Election Day,305,Governor,305,0,NaN,5b,48,...,0,0,37,9,30,7,0,0,1102,666


In [22]:
# Check number of counties matches
print(len(turnout.County.drop_duplicates()))
print(len(votes_fixed.County.drop_duplicates()))
print(len(merge.County.drop_duplicates()))
set(votes_fixed.County) - set(merge.County)

159
159
159


set()

In [23]:
# find missing precincts
for county in votes_fixed.County.unique(): 
    a = votes_fixed.loc[votes_fixed.County == county, 'Precinct_ID'].nunique()
    b = merge.loc[merge.County == county, 'Precinct_ID'].nunique()
    if a != b:
        print(county, "is missing ", a-b, "precincts")

In [11]:
set(votes_fixed.loc[votes_fixed.County=="CHATHAM", "Precinct_ID"].unique()) - \
    set(merge.loc[merge.County=="CHATHAM", "Precinct_ID"].unique())


set()

In [24]:
# find where there are too many rows
filtered_merge = merge.loc[(merge["Vote type"]=="Election Day") & (merge.Contest=="Governor")]
for county in merge.County.unique(): 
    a = turnout.loc[(turnout.County == county) & (turnout.Precinct_ID != "88888")
                    & (turnout.Precinct_ID != "99999"), 'Precinct_ID'].nunique()
    b = merge.loc[merge.County == county, 'Precinct_ID'].nunique()
    if a != b:
        print(county, "is missing ", a-b, "precincts")

GWINNETT is missing  -1 precincts
DEKALB is missing  -3 precincts


In [25]:
# Check duplicated rows
for county in merge.County.unique():
    filt = merge.loc[(merge.County == county) & (merge.Contest=="Governor") &
                     (merge["Vote type"] == "Election Day")]
    filt2 = votes_fixed.loc[(votes_fixed.County == county) & (votes_fixed.Contest=="Governor") &
                     (votes_fixed["Vote type"] == "Election Day")]
    if filt.shape[0] != filt2.shape[0]:
        print(county, filt.shape[0], filt2.shape[0])
    

CHATHAM 92 90


In [26]:
merge.to_csv("../../Data/merged_turnout_votes.csv", index=False)

# Write out with left join to find bad rows



In [ ]:
merge1 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_number"], right_on=["County","Precinct_ID"], how="left") 
merge2 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_word"], right_on=["County", "Precinct_name"], how="left") 
merge3 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_ID"], right_on=["County","Precinct_ID"], how="left")
merge4 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_ID"], right_on=["County","Precinct_name"], how="left")
merge5 = pd.merge(votes_fixed, turnout, left_on=["County", "Precinct_ID_mod"], right_on=["County","Precinct_name"], how="left")
merge_left = pd.concat([merge1, merge2, merge3, merge4, merge5], sort=False).drop_duplicates()
merge_left = merge_left.loc[merge_left.Contest == "Governor"]
merge_left = merge_left.loc[merge_left["Vote type"] == "Election Day"]
merge_left.to_csv("../../Data/leftjoin_turnout_votes.csv", index=False)

In [ ]:
# Check number of valid precincts
print(turnout.shape[0] - (turnout.loc[turnout.Precinct_ID=="88888"].shape[0] + \
                    turnout.loc[turnout.Precinct_ID=="99999"].shape[0]))

# Check number of precincts in the vote table
print(votes.loc[(votes["Contest"]=="Governor") & (votes["Vote type"]=="Election Day")].shape[0])